# IMPORT

In [ ]:
import pandas as pd

In [ ]:
from IPython.core.display import display, HTML

In [ ]:
from copy import deepcopy

In [ ]:
from seedUtils import seedUtils
import io
import os
import base64

In [ ]:
import matplotlib

import matplotlib.pyplot as plt
import matplotlib.patches as patches

import numpy as np

In [ ]:
from graphviz import Digraph

In [ ]:
import cv2

In [ ]:
%load_ext autoreload
%autoreload 2

# UTILITY FUNCTIONS

In [ ]:
def renderDatasetImg(image, title=None) :
    img = io.BytesIO()
    fig = plt.figure()
    plt.imshow(image)
    plt.axis("off")
    plt.savefig(img, format='png')
    plt.cla()
    plt.clf()
    plt.close('all')
    img.seek(0)
    plot_url = base64.b64encode(img.getvalue()).decode()
    return plot_url

In [ ]:
def renderDatasetImgMuRGB(muR, muG, muB, title=None) :
    img = io.BytesIO()
    fig = plt.figure()
    ble = np.asarray([[[muR, muG, muB]]], dtype=np.uint8)
    plt.imshow(ble)
    plt.axis("off")
    plt.savefig(img, format='png')
    plt.cla()
    plt.clf()
    plt.close('all')
    img.seek(0)
    plot_url = base64.b64encode(img.getvalue()).decode()
    return plot_url

In [ ]:
def renderChromosome(x) :
    chromosomeSplit, _ = seedUtils.splitChromosome(x)
    perimeter, bbox, colour, alpha = seedUtils.chromosome2image(x)

    img = io.BytesIO()

    fig, ax = plt.subplots()

    isShaped = chromosomeSplit[seedUtils.chromosomeOrder.index('isShaped')][0]
    if isShaped :
        poly = patches.Polygon(perimeter.transpose(), color=colour + alpha)
        ax.add_patch(poly)
        ax.scatter(perimeter[0], perimeter[1], c='black', marker='.')
        plt.xlim(bbox['x0'], bbox['x0']+bbox['width'])
        plt.ylim(bbox['y0'], bbox['y0']+bbox['height'])
    else :
        ble = np.asarray(255.0 * np.asarray(colour), dtype=np.uint8)
        ble = np.asarray([[ble]], dtype=np.uint8)
        plt.imshow(ble)

    plt.axis("off")
    plt.savefig(img, format='png')
    plt.cla()
    plt.clf()
    plt.close('all')
    img.seek(0)
    plot_url = base64.b64encode(img.getvalue()).decode()
    return plot_url

In [ ]:
def renderChromosome2(x) :
    chromosomeSplit, _ = seedUtils.splitChromosome(x)
    perimeter, bbox, colour, alpha = seedUtils.chromosome2image(x)
    isShaped = chromosomeSplit[seedUtils.chromosomeOrder.index('isShaped')][0]
    isColoured = chromosomeSplit[seedUtils.chromosomeOrder.index('isColoured')][0]

    #img = io.BytesIO()
    #fig = plt.figure()

    if isShaped :
        p2 = deepcopy(perimeter)
        p2[0] = p2[0] - bbox['x0']
        p2[1] = p2[1] - bbox['y0']
        p2[0] = p2[0] / bbox['width']
        p2[1] = p2[1] / bbox['height']
        p2 *= 99
        p2 = np.uint(p2)

        if isColoured :
            colour = list(np.asarray(colour)*255)
        else :
            colour = None

        ble = seedUtils.cv2fyV2(p2, 100, 100, 
                                bgColour=(255,255,255), 
                                perimeterColour=(0,0,0), fillColour=colour)
        #plt.imshow(ble)

    else :
        ble = np.asarray(255.0 * np.asarray(colour), dtype=np.uint8)
        ble = np.asarray([[ble]], dtype=np.uint8)
        #plt.imshow(ble)

    #plt.axis("off")
    #plt.savefig(img, format='png')
    #plt.cla()
    #plt.clf()
    #plt.close()
    #img.seek(0)
    ret, buffer = cv2.imencode('.png', ble)
    plot_url = base64.b64encode(buffer).decode()

    return plot_url

In [ ]:
def flaskStub_displayImg(img) :
    display(HTML("<img src='data:image/png;base64," + img + "'  width='100' height='100'/>"))

In [ ]:
def chromosomeStrToChromosome(dst, x) :
    ch = x.split("|")
    ch = [v.split(" ") for v in ch]
    ch = [item for sublist in ch for item in sublist]
    
    for i in range(len(ch)) :
        v = ch[i]
        try :
            dst[i] = int(v)
        except ValueError :
            dst[i] = float(v)
    

In [ ]:
def renderDataset(renderBBox=True, renderDetails=False) :
    global dataset, dataset_rgb, dataset_mask
    
    for img_idx in range(len(dataset)) :
        renderDatasetImage(img_idx, renderBBox, renderDetails)

In [ ]:
def renderDatasetImg(image, title=None) :
    ble = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    ret, buffer = cv2.imencode('.png', ble)
    plot_url = base64.b64encode(buffer).decode()

    return plot_url

In [ ]:
def renderDatasetImage(img_idx, renderBBox=True, renderDetails=False) :
    
    global dataset_rgb, dataset_mask, dataset_r, dataset_g, dataset_b, dataset_muR, dataset_muG, dataset_muB
    
    nFigures = 6 if renderDetails else 3  # for now: rgb, mask, pixel of mu colour, r, g, b,  
    
    fig, axes = plt.subplots(1, nFigures, sharex=False, sharey=False, squeeze=True)
    
    plt.subplot(1, nFigures, 1)
    refImg = deepcopy(dataset_rgb[img_idx])
    if renderBBox :
        img_bbox = getDatasetImgBBox(img_idx)
        refImg = cv2.rectangle(refImg, (img_bbox['x0'], img_bbox['y0']), 
                               (img_bbox['x0'] + img_bbox['width'], img_bbox['y0'] + img_bbox['height']), 
                               (0,255,0), 5)
    plt.imshow(refImg)
    plt.title("original")
    
    plt.subplot(1, nFigures, 2)
    plt.imshow(dataset_mask[img_idx])
    plt.title("mask")# ({0} x {1})".format(img_bbox['width'], img_bbox['height']))
    plt.axis('off')
    
    plt.subplot(1, nFigures, 3)
    ble = np.asarray([[[dataset_muR[img_idx],dataset_muG[img_idx],dataset_muB[img_idx]]]], dtype=np.uint8)
    plt.imshow(ble)
    plt.title("muRGB")
    plt.axis('off')
    
    if renderDetails :
        plt.subplot(1, nFigures, 4)
        plt.imshow(dataset_r[img_idx])
        plt.title("red")
        plt.axis('off')
        
        plt.subplot(1, nFigures, 5)
        plt.imshow(dataset_g[img_idx])
        plt.title("green")
        plt.axis('off')
        
        plt.subplot(1, nFigures, 6)
        plt.imshow(dataset_b[img_idx])
        plt.title("blue")        
        plt.axis('off')

    plt.show()

In [ ]:
def behaviourToInt(x) :
    v = [int(y) for y in x.split(';')]
    s = 0
    for i in range(len(v)) :
        s += v[i] * int(np.exp(i))
    return s

# SETTINGS

In [ ]:
baseResultsFolder = "../"

In [ ]:
resultsFolder = baseResultsFolder + "evolution/"

In [ ]:
outFolder = "evolutionNetwork"

In [ ]:
if outFolder not in os.listdir(baseResultsFolder) :
    print("creating", baseResultsFolder + outFolder)
    os.mkdir(baseResultsFolder + outFolder)

In [ ]:
outFolder = baseResultsFolder + outFolder + "/"

In [ ]:
seedUtils.initDEAP()

In [ ]:
seedUtils.loadDataset()

In [ ]:
dataset_rgb = [renderDatasetImg(img, "original") for img in seedUtils.dataset_rgb]

In [ ]:
dataset_mu = []
for i in range(len(seedUtils.dataset)) :
    muR = seedUtils.dataset_muR[i]
    muG = seedUtils.dataset_muG[i]
    muB = seedUtils.dataset_muB[i]
    dataset_mu.append(renderDatasetImgMuRGB(muR, muG, muB, "muRGB"))

# START ANALYSIS

In [ ]:
fileList = os.listdir(resultsFolder)

In [ ]:
fileList

# save the evolution in HTML format for each user

In [ ]:
for f in fileList :
    print(f)
    df = pd.read_csv(resultsFolder + f)
    print(f, df.shape)
    df['userID'] = f
    userName = f.split("_")[1]
    df['userName'] = userName
    df['behaviourInt'] = df['behaviour'].apply(lambda x : behaviourToInt(x))
    
    ble = open(outFolder + "evolution_" + f + ".html", "w")
    ble.write("<h1>" + userName + "</h1>\n")
    ble.write("<table border='1' cellpadding='5' cellspacing='10'>")
    ble.write("<tr><td>Loop</td><td>Chromosome</td><td>Thresholds</td><td>Classified Dataset</td></tr>")
    for i in range(len(df)) :
        
        # loop (for network mixing)
        html = "<tr><td>" + str(i) + "</td>"
        
        g = df.iloc[i]['genotype']
        x = [-1 for _ in range(seedUtils.chromosomeLen)]
        
        chromosomeStrToChromosome(x, g)
        img = renderChromosome2(x)
        d = seedUtils.splitChromosome(x)[1]
        
        # chromosome
        html += "</td><td><img src='data:image/png;base64," + img + "'  width='100' height='100'/></td>"
        
        chromosomeBehaviour = [int(x) for x in df.iloc[i]['behaviour'].split(";")]
        #print(sum(chromosomeBehaviour))
        
        # thresholds
        html += "<td>Colour:" + str(d['colourMatchingOverlap'][0]) + " Shape:" + str(d['shapeMatchingOverlap'][0])  + "</td>"
        
        # classified dataset        
        html += "<td>"
        
        for j in range(len(chromosomeBehaviour)) :
            if chromosomeBehaviour[j] == 1 :
                html += "<img src='data:image/png;base64," + dataset_rgb[j] + "'  width='50' height='50'/>"

        html += "</td></tr>"
        
        ble.write(html + "\n")
    ble.write("</table>")
    ble.close()
print("done! now print!")

# IDENTIFY EVOLUTION INTERSECTIONS

In [ ]:
megaDF = pd.DataFrame()
for f in fileList :
    df = pd.read_csv(resultsFolder + f)
    print(f, df.shape)
    df['userID'] = f
    df['userName'] = f.split("_")[1]
    df['behaviourInt'] = df['behaviour'].apply(lambda x : behaviourToInt(x))
    megaDF = megaDF.append(df, ignore_index=True)

In [ ]:
megaDF.shape

In [ ]:
behaviours = list(megaDF['behaviourInt'].drop_duplicates())

In [ ]:
for b in behaviours :
    miniDf = megaDF[megaDF['behaviourInt'] == b]
    users = list(miniDf['userID'].drop_duplicates())
    if len(users) > 1 :
        print("intersection at behaviourInt", b)
        for u in users :
            loops = list(miniDf[miniDf['userID'] == u]['loop'])
            print("\t", u, "Loops:", ";".join([str(x) for x in loops]))